# Mann-Whitney U test
### A real-world example from scratch
## Introduction
The Mann-Whitney U test is employed to evaluate significant differences between two independent groups. The process entails ranking all data points collectively, irrespective of their group affiliation. Subsequently, the U statistic, which signifies the sum of ranks for one group, is computed. This test is non-parametric, indicating it doesn't hinge on the assumption of normal distribution. It acts as an alternative to the independent t-test and is known by monikers such as Wilcoxon Rank Sum Test or Wilcoxon–Mann–Whitney test. The following is an illustrative example in a real-world scenario.
### Hypotheses
- Null hypotheses: median1 = median2
- Alternative hypotheses: median1 ≠ median2
### Assumptions
**Independence**: The observations in one group are assumed to be independent of the observations in the other group.

**Ordinal Scale**: The data should be measured on an ordinal scale, which means the values can be ranked. This is necessary because the test is based on the ranks of the observations.

**Random Sampling**: The data should be collected through a random sampling process.

**Equal Shape of Distributions**: The Mann-Whitney U test assumes that the shapes of the distributions for both groups are similar.
### Output
- U statistics
- p value
### Skewness
Pearson's second coefficient can be used to evaluate the skewness of data.
Pearson's second coefficient = 3*(mean - median)/standard deviation
- the skewness is between -0.5 & 0.5, the data are nearly symmetrical.
- the skewness is between -1 & -0.5 (negative skewed) or between 0.5 & 1(positive skewed), the data are slightly skewed.
- the skewness is lower than -1 (negative skewed) or greater than 1 (positive skewed), the data are extremely skewed.

## Generate skewed data
Firstly, let's generate two skewed data samples. For example, the data in reality can the efficiency (shunt resistance, fill factor, power) of solar cells from test A and test B. The difference between group A and B can be the production machines, supttering material, or substract holders.
Let's use Gamma distribution to simulate the efficiencies of solar cells. The probability density for the Gamma distribution is as follows:
\begin{equation}
      p(x) = x^{k-1}\frac{e^{-x/\theta}}{\theta^k\Gamma(k)}
\end{equation}
where $k$ is the shape and $\theta$ the scale,
and $\Gamma$ is the Gamma function. Suppose there are 50 samples in each test group.

In [34]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import scipy.special as sps
from scipy import stats

n_sample = 50
k, theta = 2, 1 # mean and width
groupA = 18 - np.random.standard_gamma(k, n_sample) # Generate data for group A
groupB = 17 - np.random.standard_gamma(k, n_sample) # Generate data from group B

# Display data
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.03, horizontal_spacing= 0.01) 
# The first chart display the data itself, the second one show the distribution.
fig.add_trace(go.Box(x=groupA, name = 'groupA', marker_color = 'rgba(255, 127, 14, 0.4)', line_color='rgb(255, 127, 14)',
                     boxpoints='all', jitter=0.3, pointpos=-1.8, showlegend=False, line_width=1), row =1, col=1)
fig.add_trace(go.Box(x=groupB, name = 'groupB', marker_color = 'rgba(148, 103, 189, 0.4)', line_color='rgb(148, 103, 189)',
                     boxpoints='all', jitter=0.3, pointpos=-1.8, showlegend=False, line_width=1), row =1, col=1)
fig.add_trace(go.Histogram(x=groupA, name = 'groupA', xbins = dict(size=0.3), histnorm='probability density',
                           marker_line_color = 'rgb(255, 127, 14)', marker_color = 'rgba(255, 127, 14, 0.4)',
                           marker_line_width=1), row=2, col=1)
fig.add_trace(go.Histogram(x=groupB, name = 'groupB', xbins = dict(size=0.3), histnorm='probability density',
                           marker_line_color = 'rgb(148, 103, 189)', marker_color = 'rgba(148, 103, 189, 0.4)',
                           marker_line_width=1), row=2, col=1)

# Add probability curve
xA = sorted(groupA)
xB = sorted(groupB)
xA = np.array(xA)
xB = np.array(xB)
def pdf_gamma(x, theta, k):
    y = x**(k-1) * ((np.exp(-x/theta))/(sps.gamma(k) * (theta**k)))
    return y
yA = pdf_gamma(18 - xA, theta, k)
yB = pdf_gamma(17-xB, theta, k)

fig.add_trace(go.Scatter(x=xA, y=yA, mode='lines', name = 'groupA', line=dict(color='rgb(255, 127, 14)', width=1.5, dash='solid'))
              , row=2, col=1)
fig.add_trace(go.Scatter(x=xB, y=yB, mode='lines', name = 'groupB', line=dict(color='rgb(148, 103, 189)', width=1.5, dash='dash'))
              , row=2, col=1)
# Overlay both histograms
fig.update_layout(title='Efficiency samples and distributions for two groups',
                  xaxis2 = dict(title="Efficiency"),
                  barmode='overlay', height=300, width=600, margin = dict(l=20, r=2, t=50, b=2))
fig.show()

## Step by step case study
### Step 1: Combine data and calculate ranks for both groups

In [35]:
df_AB = pd.DataFrame({'groupA': groupA, 'groupB': groupB})
# Reshape df_AB into long format
df_AB2 = pd.melt(df_AB, value_vars=['groupA', 'groupB'], var_name='Group', value_name='Eff')
df_AB3 = df_AB2.copy()
df_AB3.loc[:, 'Rank'] = df_AB3['Eff'].rank()

### Step 2: Calculate the rank sums respectively for A and B

In [36]:
cate_names = df_AB3['Group'].unique() # list group names
rankSum = df_AB3.groupby(by=['Group'])['Rank'].sum()
print(rankSum)
[rankSumA, rankSumB] = rankSum

Group
groupA    2795.0
groupB    2255.0
Name: Rank, dtype: float64


### Step 3: Calculate the U-values

In [37]:
[nA, nB] = df_AB3.groupby(by=['Group'])['Rank'].count()
U_A = nA*nB + nA*(nA+1)/2 - rankSumA
U_B = nA*nB + nB*(nB+1)/2 - rankSumB
print('UA = ', U_A, ', UB = ', U_B)
U_Wert = min(U_A, U_B)
print('U_Wert = ', U_Wert)

UA =  980.0 , UB =  1520.0
U_Wert =  980.0


### Step 4: Check if there are tied ranks

In [38]:
df_AB3['IsRankTied'] = df_AB3['Rank'].duplicated(keep=False)
df_AB3['IsRankTied'].value_counts()

False    100
Name: IsRankTied, dtype: int64

### Step 5: Calculate the p-value
The U statistic is assumed to yield a normal distribution under the null hypothesis in the Mann-Whitney U test. The standardization process (normalising) is often done to facilitate the comparison of the U statistic to a standard normal distribution. The resulting z-value is also approximately normally distributed under the null hypothesis.

In [39]:
# Expected value of U
U_ex = nA*nB/2

# When there is no tied ranks
# Standard error of U
U_sigma = np.sqrt(nA*nB*(nA+nB+1)/12)

# when there are tied ranks
# Standard error of U with tie correction
_, ties = np.unique(df_AB3['Rank'], return_counts=True, axis=-1)
tieSum = (ties ** 3 - ties).sum() # this procedure will help detact if the tied-ranks exist.
U_sigmaTie = np.sqrt( (nA*nB/12) * ( (nA+nB+1) - tieSum/((nA+nB)*(nA+nB-1)) ) )

# Calculate z-value
z = (U_Wert - U_ex)/U_sigma

# Calculate p-value
p = stats.norm.sf(abs(z))
print('p-vlaue =', p)

p-vlaue = 0.03134869814386435


For a one-tailed hypothesis test, the p-value is 0.001266; for a two-tailed hypothesis test, the p-value should be multiplied by 2, yielding p-value = 0.002532. In both cases, the p-value is below the 5% threshold. Consequently, we reject the null hypothesis asserting that group A and group B share the same distribution. Thus, we conclude that group A and group B in our case are significantly different.

## Robustness Verification Through Repeated Hypothesis Tests
Nonetheless, a single hypothesis test may lack precision in real-world scenarios due to sampling variability. A pragmatic approach involves conducting repeated hypothesis tests, for instance, 100 times. This allows us to compare the statistical outcomes between hypothesis tests of group A vs. group A (or group B vs. group B) and group A vs. group B.

### Wrap the algorithm to a function

In [40]:
import pandas as pd
import numpy as np
from scipy import stats

def mann_whitney_u_test(groupA, groupB, tails=2):
    # Combine groups into a DataFrame
    df_AB = pd.DataFrame({'groupA': groupA, 'groupB': groupB})
    
    # Reshape df_AB into long format
    df_AB2 = pd.melt(df_AB, value_vars=['groupA', 'groupB'], var_name='Group', value_name='Eff')
    df_AB3 = df_AB2.copy()
    df_AB3.loc[:, 'Rank'] = df_AB3['Eff'].rank()
    
    # Calculate rank sums
    rankSum = df_AB3.groupby(by=['Group'])['Rank'].sum()
    [rankSumA, rankSumB] = rankSum
    [nA, nB] = df_AB3.groupby(by=['Group'])['Rank'].count()
    
    # Calculate U values
    U_A = nA*nB + nA*(nA+1)/2 - rankSumA
    U_B = nA*nB + nB*(nB+1)/2 - rankSumB
    U_Wert = min(U_A, U_B)
    
    # Expected value of U
    U_ex = nA*nB/2
    
    # Calculate standard error of U
    _, ties = np.unique(df_AB3['Rank'], return_counts=True, axis=-1)
    tieSum = (ties ** 3 - ties).sum()
    U_sigmaTie = np.sqrt((nA * nB / 12) * ((nA + nB + 1) - tieSum / ((nA + nB) * (nA + nB - 1))))
    
    # Calculate z-value
    z = (U_Wert - U_ex) / U_sigmaTie
    
    # Calculate p-value
    if tails == 2:
        p = 2 * stats.norm.sf(abs(z))
    else:
        p = stats.norm.sf(abs(z))
    
    return p

### Calculate p-values for comparison

In [41]:
p_values_mannwhitneyu_AA = []
p_values_mannwhitneyu_BB = []
p_values_mannwhitneyu_AB = []
n_test = 100
n_sample2 = 40  

for _ in range(n_test):
    # Generate samples for the repeated tests
    resampleA = np.random.choice(groupA, size=n_sample2, replace=False) # set replace to True if bootstrapping is desired in your case.
    resampleA2 = np.random.choice(groupA, size=n_sample2, replace=False)
    resampleB = np.random.choice(groupB, size=n_sample2, replace=False)
    resampleB2 = np.random.choice(groupB, size=n_sample2, replace=False)

    # Calculate the mann whitney u tests for comparsion
    p_AB = mann_whitney_u_test(resampleA, resampleB, tails=2)
    p_AA = mann_whitney_u_test(resampleA, resampleA2, tails=2)
    p_BB = mann_whitney_u_test(resampleB, resampleB2, tails=2)

    # save results
    p_values_mannwhitneyu_AB.append(p_AB)
    p_values_mannwhitneyu_AA.append(p_AA)
    p_values_mannwhitneyu_BB.append(p_BB)

# Combine and reshape data for plot
df_combine = pd.DataFrame({'p-value': p_values_mannwhitneyu_AA + p_values_mannwhitneyu_BB + p_values_mannwhitneyu_AB,
                           'Comparison': ['A vs A'] * n_test + ['B vs B'] * n_test + ['A vs B'] * n_test})

df_combine


,p-value,Comparison
0,0.383757,A vs A
1,0.312231,A vs A
2,0.603268,A vs A
3,0.512820,A vs A
4,0.479329,A vs A
...,...,...
295,0.045341,A vs B
296,0.067508,A vs B
297,0.146224,A vs B
298,0.022012,A vs B


### Show comparsion results

In [42]:
import plotly_express as px
# plot the results in a facet graph
fig2 = make_subplots(rows=1, cols=3,vertical_spacing=0.03, horizontal_spacing=.02, 
                shared_xaxes= True, shared_yaxes= True) 
batch_colors = ['rgb(31, 119, 180)', 'rgb(255, 127, 14)', 'rgb(44, 160, 44)']

fig2 = px.histogram(df_combine, x="p-value", color= 'Comparison',
                histnorm='probability', color_discrete_sequence = batch_colors,
                facet_col="Comparison")    
fig2.add_vline(x=0.05,  line_width=1, line_dash="dash", line_color='rgb(214, 39, 40)')
fig2.update_traces(xbins=dict(start=0, size = 0.05))
fig2.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig2.add_annotation(text= '0.05', x=0.05, y=-0.15,  font_color= 'rgb(214, 39, 40)',
                xref="x", yref="paper", font_size=12, showarrow=False)
fig2.add_annotation(text= '0.05', x=0.05, y=-0.15,  font_color= 'rgb(214, 39, 40)',
                xref="x2", yref="paper", font_size=12, showarrow=False)
fig2.add_annotation(text= '0.05', x=0.05, y=-0.15,  font_color= 'rgb(214, 39, 40)',
                xref="x3", yref="paper", font_size=12, showarrow=False)
fig2.update_layout(title_text=f"<b>Distribution of p-values for comparison", 
                        height=300, width=600, template='plotly', showlegend=False,
                        bargap=0.1, margin = dict(l=20, r=2, t=50, b=2))
fig2.show()

### Interpretation
If the samples originate from the same distribution, the probability distribution is expected to resemble (A vs A) or (B vs B). However, distinct differences in distribution shapes emerge when comparing (A vs A) or (B vs B) to (A vs B). Consequently, the observed dissimilarities strongly suggest a significant distinction between groupA and groupB.